In [1]:
from __future__ import print_function
#import sys
#if len(sys.argv) != 4:
#    print('Usage:')
#    print('python train.py datacfg cfgfile weightfile')
#    exit()

import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable

import dataset
import random
import math
import os
from utils import *
from cfg import parse_cfg
from region_loss import RegionLoss
from darknet import Darknet
from models.tiny_yolo import TinyYoloNet

In [2]:
# Training settings
datacfg       = 'cfg/voc.data'#sys.argv[1]
#cfgfile       = 'cfg/tiny-yolo-voc.cfg'#sys.argv[2]
cfgfile       = 'cfg/tiny-yolo-hackathon.cfg'
#weightfile    = 'darknet19_448.conv.23'#sys.argv[3]
weightfile    = 'yolov2-tiny-voc.weights'

data_options  = read_data_cfg(datacfg)
net_options   = parse_cfg(cfgfile)[0]

trainlist     = data_options['train']
testlist      = data_options['valid']
backupdir     = data_options['backup']
nsamples      = file_lines(trainlist)
gpus          = data_options['gpus']  # e.g. 0,1,2,3
ngpus         = len(gpus.split(','))
num_workers   = int(data_options['num_workers'])

batch_size    = int(net_options['batch'])
max_batches   = int(net_options['max_batches'])
learning_rate = float(net_options['learning_rate'])
momentum      = float(net_options['momentum'])
decay         = float(net_options['decay'])
steps         = [float(step) for step in net_options['steps'].split(',')]
scales        = [float(scale) for scale in net_options['scales'].split(',')]

#Train parameters
max_epochs    = max_batches*batch_size/nsamples+1
use_cuda      = True
seed          = int(time.time())
eps           = 1e-5
save_interval = 10  # epoches
dot_interval  = 70  # batches

# Test parameters
conf_thresh   = 0.25
nms_thresh    = 0.4
iou_thresh    = 0.5

if not os.path.exists(backupdir):
    os.mkdir(backupdir)
    
###############
torch.manual_seed(seed)
if use_cuda:
    os.environ['CUDA_VISIBLE_DEVICES'] = gpus
    torch.cuda.manual_seed(seed)

model       = Darknet(cfgfile)
region_loss = model.loss

In [3]:
model.seen

0

In [4]:
model.load_weights(weightfile)
#model.print_network()

/home/paperspace/yolo2_onnx/cfg.py:179: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  conv_model.weight.data.copy_(torch.from_numpy(buf[start:start+num_w])); start = start + num_w #20180615 modified for <= torch v0.3.0 post4
/home/paperspace/yolo2_onnx/cfg.py:158: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  conv_model.weight.data.copy_(torch.from_numpy(buf[start:start+num_w])); start = start + num_w #20180615 modified for <= torch v0.3.0 post4


In [5]:
model.seen

2566400

In [6]:
model.seen = 0

In [7]:
region_loss.seen  = model.seen
processed_batches = model.seen/batch_size

init_width        = model.width
init_height       = model.height
init_epoch        = model.seen/nsamples 

In [8]:
cur_model = model
train_loader = torch.utils.data.DataLoader(
    dataset.listDataset(trainlist, shape=(init_width, init_height),
                   shuffle=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]), 
                   train=True, 
                   seen=cur_model.seen,
                   batch_size=batch_size,
                   num_workers=num_workers),
    batch_size=batch_size, shuffle=False)

In [9]:
train_loader

In [10]:
(data, target) = next(iter(train_loader))

In [11]:
data, target = Variable(data), Variable(target)

In [12]:
output = model(data)

In [ ]:
region_loss(output, target)

In [11]:
??torch.utils.data.DataLoader

In [8]:
kwargs = {'num_workers': num_workers, 'pin_memory': True} if use_cuda else {}
test_loader = torch.utils.data.DataLoader(
    dataset.listDataset(testlist, shape=(init_width, init_height),
                   shuffle=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]), train=False),
    batch_size=batch_size, shuffle=False, **kwargs)

if use_cuda:
    if ngpus > 1:
        model = torch.nn.DataParallel(model).cuda()
    else:
        model = model.cuda()

params_dict = dict(model.named_parameters())
params = []
for key, value in params_dict.items():
    if key.find('.bn') >= 0 or key.find('.bias') >= 0:
        params += [{'params': [value], 'weight_decay': 0.0}]
    else:
        params += [{'params': [value], 'weight_decay': decay*batch_size}]
optimizer = optim.SGD(model.parameters(), lr=learning_rate/batch_size, momentum=momentum, dampening=0, weight_decay=decay*batch_size)

def adjust_learning_rate(optimizer, batch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = learning_rate
    for i in range(len(steps)):
        scale = scales[i] if i < len(scales) else 1
        if batch >= steps[i]:
            lr = lr * scale
            if batch == steps[i]:
                break
        else:
            break
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr/batch_size
    return lr

def train(epoch):
    #pdb.set_trace()
    global processed_batches
    t0 = time.time()
    if ngpus > 1:
        cur_model = model.module
    else:
        cur_model = model
    train_loader = torch.utils.data.DataLoader(
        dataset.listDataset(trainlist, shape=(init_width, init_height),
                       shuffle=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                       ]), 
                       train=True, 
                       seen=cur_model.seen,
                       batch_size=batch_size,
                       num_workers=num_workers),
        batch_size=batch_size, shuffle=False, drop_last=True, **kwargs)

    lr = adjust_learning_rate(optimizer, processed_batches)
    logging('epoch %d, processed %d samples, lr %f' % (epoch, epoch * len(train_loader.dataset), lr))
    model.train()
    t1 = time.time()
    avg_time = torch.zeros(9)
    for batch_idx, (data, target) in enumerate(train_loader):
        t2 = time.time()
        adjust_learning_rate(optimizer, processed_batches)
        processed_batches = processed_batches + 1
        #if (batch_idx+1) % dot_interval == 0:
        #    sys.stdout.write('.')

        if use_cuda:
            data = data.cuda()
            #target= target.cuda()
        t3 = time.time()
        data, target = Variable(data), Variable(target)
        t4 = time.time()
        optimizer.zero_grad()
        t5 = time.time()
        output = model(data)
        t6 = time.time()
        region_loss.seen = region_loss.seen + data.data.size(0)
        loss = region_loss(output, target)
        t7 = time.time()
        loss.backward()
        t8 = time.time()
        optimizer.step()
        t9 = time.time()
        if False and batch_idx > 1:
            avg_time[0] = avg_time[0] + (t2-t1)
            avg_time[1] = avg_time[1] + (t3-t2)
            avg_time[2] = avg_time[2] + (t4-t3)
            avg_time[3] = avg_time[3] + (t5-t4)
            avg_time[4] = avg_time[4] + (t6-t5)
            avg_time[5] = avg_time[5] + (t7-t6)
            avg_time[6] = avg_time[6] + (t8-t7)
            avg_time[7] = avg_time[7] + (t9-t8)
            avg_time[8] = avg_time[8] + (t9-t1)
            print('-------------------------------')
            print('       load data : %f' % (avg_time[0]/(batch_idx)))
            print('     cpu to cuda : %f' % (avg_time[1]/(batch_idx)))
            print('cuda to variable : %f' % (avg_time[2]/(batch_idx)))
            print('       zero_grad : %f' % (avg_time[3]/(batch_idx)))
            print(' forward feature : %f' % (avg_time[4]/(batch_idx)))
            print('    forward loss : %f' % (avg_time[5]/(batch_idx)))
            print('        backward : %f' % (avg_time[6]/(batch_idx)))
            print('            step : %f' % (avg_time[7]/(batch_idx)))
            print('           total : %f' % (avg_time[8]/(batch_idx)))
        t1 = time.time()
    print('')
    t1 = time.time()
    logging('training with %f samples/s' % (len(train_loader.dataset)/(t1-t0)))
    if (epoch+1) % save_interval == 0:
        logging('save weights to %s/%06d.weights' % (backupdir, epoch+1))
        cur_model.seen = (epoch + 1) * len(train_loader.dataset)
        cur_model.save_weights('%s/%06d.weights' % (backupdir, epoch+1))

def test(epoch):
    def truths_length(truths):
        for i in range(50):
            if truths[i][1] == 0:
                return i

    model.eval()
    if ngpus > 1:
        cur_model = model.module
    else:
        cur_model = model
    num_classes = cur_model.num_classes
    anchors     = cur_model.anchors
    num_anchors = cur_model.num_anchors
    total       = 0.0
    proposals   = 0.0
    correct     = 0.0

    for batch_idx, (data, target) in enumerate(test_loader):
        if use_cuda:
            data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data).data
        all_boxes = get_region_boxes(output, conf_thresh, num_classes, anchors, num_anchors)
        for i in range(output.size(0)):
            boxes = all_boxes[i]
            boxes = nms(boxes, nms_thresh)
            truths = target[i].view(-1, 5)
            num_gts = truths_length(truths)
     
            total = total + num_gts
    
            for i in range(len(boxes)):
                if boxes[i][4] > conf_thresh:
                    proposals = proposals+1

            for i in range(num_gts):
                box_gt = [truths[i][1], truths[i][2], truths[i][3], truths[i][4], 1.0, 1.0, truths[i][0]]
                best_iou = 0
                best_j = -1
                for j in range(len(boxes)):
                    iou = bbox_iou(box_gt, boxes[j], x1y1x2y2=False)
                    if iou > best_iou:
                        best_j = j
                        best_iou = iou
                if best_iou > iou_thresh and boxes[best_j][6] == box_gt[6]:
                    correct = correct+1

    precision = 1.0*correct/(proposals+eps)
    recall = 1.0*correct/(total+eps)
    fscore = 2.0*precision*recall/(precision+recall+eps)
    logging("precision: %f, recall: %f, fscore: %f" % (precision, recall, fscore))

In [9]:
evaluate = False
if evaluate:
    logging('evaluating ...')
    test(0)
else:
#    for epoch in range(int(init_epoch), int(max_epochs)): 
    for epoch in range(int(init_epoch), int(1)): 
        train(epoch)
        test(epoch)

2018-07-13 08:36:52 epoch 0, processed 0 samples, lr 0.000100


/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/tensor.py:305: UserWarning: self and other not broadcastable, but have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add(other)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/tensor.py:321: UserWarning: self and other not broadcastable, but have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.mul(other)


64: nGT 3, recall 0, proposals 100, loss: x 1021.271912, y 1926.883789, w 20105.025391, h 21047.841797, conf 4094.981934, cls 0.000000, total 48196.003906
128: nGT 6, recall 0, proposals 238, loss: x 973.403259, y 1836.299927, w 27365.906250, h 27006.046875, conf 3231.294922, cls 0.000000, total 60412.953125
192: nGT 5, recall 1, proposals 215, loss: x 855.554993, y 1688.666016, w 6209.292969, h 6484.882324, conf 2077.793213, cls 0.000000, total 17316.189453
256: nGT 5, recall 0, proposals 99, loss: x 664.151428, y 1526.524170, w 22488.265625, h 24016.718750, conf 1161.247192, cls 0.000000, total 49856.906250
320: nGT 3, recall 0, proposals 214, loss: x 531.396179, y 1290.569580, w 7340.053711, h 6874.397461, conf 673.365906, cls 0.000000, total 16709.783203
384: nGT 6, recall 0, proposals 15, loss: x 404.449371, y 1144.697021, w 10856.240234, h 12685.431641, conf 395.857880, cls 0.000000, total 25486.675781
448: nGT 3, recall 2, proposals 165, loss: x 339.718872, y 985.537964, w 11038

/home/paperspace/yolo2_onnx/utils.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_confs = torch.nn.Softmax()(Variable(output[5:5+num_classes].transpose(0,1))).data


2018-07-13 08:37:55 precision: 0.000000, recall: 0.000000, fscore: 0.000000
